In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
#from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama

embeddings = OllamaEmbeddings(model='nomic-embed-text:v1.5')

path_books = "./vector-store/books"
db = FAISS.load_local(path_books, embeddings, allow_dangerous_deserialization=True)

retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 5

In [2]:
question = "Como devemos tratar novos crentes?"

In [6]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama

# RAG
template = """Responda a pergunta abaixo com base no contexto fornecido.

Contexto: {context}

Questão: {question}

Responda em português da seguinte formato:

Resposta: Aqui a resposta

Contexto utilizado: "Citação direta do contexto"
"""

prompt = ChatPromptTemplate.from_template(template)

llm = Ollama(model="mistral:7b", temperature=0)

final_rag_chain = (
    {"context": retriever, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

' Resposta: Tratar novos crentes com paciência, amor e ensinamento. Não devemos pressioná-los ou exigir de eles o que ainda não estão prontos para fazer. Em vez disso, nos esforçamos em ajudá-los a crescer na fé, fornecendo-lhes instrução básica e apoio espiritual. Além disso, devemos ser exemplos vivos de como viver uma vida cristã, demonstrando amor, compaixão e humildade.\n\nContexto utilizado: "Prossigam nos seus encargos jovens, regozijando-se com o trabalho que têm a fazer, pois isso implica fazer o bem. Eu tinha um jardinzinho e, um dia, plantei mostarda e agrião. Na manhã seguinte, fui ver se já estava brotando e não fiquei satisfeito de ter que esperar a época devida. Revolvi a terra do cantiero e ouso dizer que impedi o crescimento da semente por causa da minha pressa. É bem possível que os professores cometam a mesma tolice de querer apressar o não crente, com a expectativa de colher amanhã o que semearam hoje. O fruto imediato pode vir, porque Deus opera maravilhosamente, m

In [2]:
# Multi Query: Different Perspectives
template = """Você é um assistente de modelo de linguagem de IA. Sua tarefa é gerar cinco
diferentes versões da pergunta do usuário para recuperar documentos relevantes de um vetor
base de dados. Ao gerar múltiplas perspectivas sobre a pergunta do usuário, seu objetivo é ajudar
o usuário supera algumas das limitações da busca por similaridade baseada em distância.
Forneça essas perguntas alternativas separadas por novas linhas na língua da pergunta original. 
Pergunta original: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)



# LLM
llm = Ollama(model="mistral:7b")

generate_queries = (
    prompt_perspectives 
    | llm #ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [3]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve

retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

/home/cristian/miniconda3/envs/ollama/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


13

In [4]:
from operator import itemgetter
#from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama

# RAG
template = """Responda a seguinte questão baseado nesse contexto:

{context}

Questão: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

#llm = ChatOpenAI(temperature=0)
llm = Ollama(model="mistral:7b", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

' Resposta: De acordo com o texto bíblico citado, Jesus respondeu a essa questão dizendo que as pessoas deveriam amar e ensinar os novos crentes, assim como ele mesmo fez. Ele também enfatizou a importância de ser pacientes e gentis com eles, pois eles ainda estavam aprendendo e crescendo em sua fé. (Mateus 18:6)\n\nNo texto citado especificamente, Jesus respondeu aos discípulos sobre como tratar um irmão que se enredou na sua fé, dizendo-lhes: "Então vai, e deixe-o estar; vai, e teu irmão pecar contra ti. Seleciona sete homens da igreja, e vai com eles, simplesmente, e ame seu irmão na presença dos seus. Então o retornarás, e tu também voltarás, pois eu estou no meio de você; se também fora outro, eu te ajudaria; mas vosso irmão pecador é teu." (Lucas 17:3-4)\n\nAssim, a mensagem bíblica é de amor, paciência e ensino para os novos crentes.'

In [16]:
relevant_documents = retriever.get_relevant_documents(question)
for row in relevant_documents:
    page_content = row.page_content
    source = row.metadata['source']
    page = row.metadata['page']

    

    # Agora você pode fazer o que quiser com esses dados.
    # Por exemplo, você pode imprimi-los:
    print(f"""
Page Content: 
{page_content.replace('	',' ')}

Source: 
{source} 

Page:
{page}

""")


Page Content: 
8
O tipo de obreiro que Deus procura
Que tipo de homem o Mestre quer usar? Certamente é aquele que é um trabalhador.
O  grande  último dia  será  muito  difícil  para  aquele  que  não  trabalhar  arduamente  em  seu
ministério, pois terá de responder pelo seu ócio. Quem deseja uma vida fácil nunca deveria
pensar em ocupar o púlpito cristão. O púlpito não é o lugar dessa pessoa, e, quando ela ali
chegar, o único conselho que posso lhe dar é que saia dessa posição o mais rápido possível. Se,
por acaso, tal pessoa não deixar o púlpito, lembrá-la-ei das palavras de Jeú em relação a Jezabel:
“Jogai-a daí para baixo” (2Rs 9.33) — e penso que isso se aplica ao ministro preguiçoso. O
preguiçoso não tem direito ao púlpito. Ele é um instrumento de Satanás para condenar a alma dos
homens. Inteligente ministério exige esforço mental; o pregador deve usar a inteligência em seu
ensino, ler e estudar a fim de manter sua mente saudável. Não deve aborrecer as pessoas,
dizendo-lhes a ve